In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df =spark.read.csv('BigBasket_Products.csv',header=True)
df.createOrReplaceTempView("BigBasket")


In [5]:
query="select * from BigBasket limit 10"
s=spark.sql(query)

In [6]:
s.show()

+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               index|             product|            category|        sub_category|            brand|          sale_price|        market_price|                type|              rating|         description|
+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   1|Garlic Oil - Vege...|    Beauty & Hygiene|           Hair Care|Sri Sri Ayurveda |                 220|                 220|    Hair Oil & Serum|                 4.1|This Product cont...|
|                   2|Water Bottle - Or...|Kitchen, Garden &...|Storage & Accesso...|       Mastercook|                 180|                 180|Water & Fridge Bo..

In [10]:
#fetch max number of products
query = " select category, count(category) as count from BigBasket group by category order by count desc "
s=spark.sql(query)

In [11]:
s.show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|    Beauty & Hygiene| 7867|
|Gourmet & World Food| 4689|
|Kitchen, Garden &...| 3580|
|Snacks & Branded ...| 2813|
|Foodgrains, Oil &...| 2676|
|Cleaning & Household| 2675|
|           Beverages|  885|
|Bakery, Cakes & D...|  851|
|           Baby Care|  610|
| Fruits & Vegetables|  557|
|   Eggs, Meat & Fish|  350|
|            warnings|   32|
| or want to help ...|   25|
| processed and pa...|   22|
| produce & availa...|   22|
|     Sweet Pani Puri|   21|
|            proteins|   19|
|               Soaps|   16|
| Ocean Glass Publ...|   16|
|  dips and conserves|   16|
+--------------------+-----+
only showing top 20 rows



In [14]:
#fetch max number of brand
query = " select brand, count(brand) as count from BigBasket group by  brand order by count desc "
s=spark.sql(query)
s.show()

+----------------+-----+
|           brand|count|
+----------------+-----+
|          Fresho|  638|
|        bb Royal|  539|
|         BB Home|  428|
|              DP|  250|
|Fresho Signature|  171|
|        bb Combo|  168|
|            Amul|  153|
|         INATUR |  146|
|        Himalaya|  141|
|           Dabur|  138|
|        GoodDiet|  134|
|           Cello|  124|
|            Nike|  124|
|           Iveo |  118|
|        BIOTIQUE|  117|
|    Aroma Magic |  109|
|         Colgate|  107|
|  Organic Tattva|  106|
|    Loreal Paris|  104|
|       Britannia|  104|
+----------------+-----+
only showing top 20 rows



In [31]:
#fetch max number of profit
query = """ SELECT 
    category, 
    COALESCE(SUM(market_price), 0) AS total_market_price, 
    COALESCE(SUM(sale_price), 0) AS total_sale_price, 
    COALESCE(SUM(market_price - sale_price), 0) AS total_profit,
    CASE
        WHEN COALESCE(SUM(market_price - sale_price), 0) = 0 THEN 'Hi'
        ELSE 'no hi'
    END AS cc
FROM 
    BigBasket 
GROUP BY 
    category 
ORDER BY 
    total_profit DESC;



 """
s=spark.sql(query)
s.show()

+--------------------+------------------+------------------+------------------+-----+
|            category|total_market_price|  total_sale_price|      total_profit|   cc|
+--------------------+------------------+------------------+------------------+-----+
|    Beauty & Hygiene|3882642.2199999997| 3293749.239999999| 588892.9799999997|no hi|
|Kitchen, Garden &...|         2361574.4|1816938.1200000003|         544636.28|no hi|
|Gourmet & World Food|1680643.9500000002|1499765.3099999998|180878.64000000004|no hi|
|Foodgrains, Oil &...|          615833.0|516916.22999999975| 98916.76999999999|no hi|
|Cleaning & Household|         701150.35| 605013.0899999999| 96137.26000000001|no hi|
|           Baby Care|          364020.0| 326317.1699999999| 37702.82999999999|no hi|
|Snacks & Branded ...|          395971.5|364505.07999999996|          31466.42|no hi|
|           Beverages|          240927.0|212186.67000000007|28740.329999999998|no hi|
|   Eggs, Meat & Fish|         114042.42|         1011

The `COALESCE` function in SQL is used to return the first non-null expression among its arguments. It takes multiple arguments and returns the first non-null value from the provided arguments. If all the arguments are null, it returns null.

The syntax for the `COALESCE` function is as follows:

```sql
COALESCE(value1, value2, ..., valueN)
```

Here, `value1`, `value2`, and so on represent the values that you want to check for null. The function will return the first non-null value from the provided arguments. If all the values are null, it will return null.

In the context of your query, `COALESCE` is used to handle cases where the `market_price` or `sale_price` might be `NULL`. By using `COALESCE` with a default value of 0, you ensure that if any of the values are `NULL`, they are treated as 0 during the summation. This prevents any potential errors that might arise from aggregating functions such as `SUM` when dealing with `NULL` values.